In [7]:
import pandas as pd
from pandas import DataFrame

Cars = pd.read_csv('../data/Cars93.csv')

In [8]:
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

model = smf.ols(formula = 'Price ~ EngineSize + RPM + Weight + Length', data = Cars)
result = model.fit()
result.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.563
Model:                            OLS   Adj. R-squared:                  0.543
Method:                 Least Squares   F-statistic:                     28.34
Date:                Mon, 04 Apr 2022   Prob (F-statistic):           3.93e-15
Time:                        22:25:50   Log-Likelihood:                -303.89
No. Observations:                  93   AIC:                             617.8
Df Residuals:                      88   BIC:                             630.4
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -45.4934     14.654     -3.104      0.003     -74.616     -16.371
EngineSize     4.5091      1.381      3.266      0.002       1.765       7.253
RPM            0.0070      0.001      5.139      0.000       0.004       0.010
Weight         0.0079      0.002      3.255      0.002       0.003       0.013
Length        -0.0457      0.083     -0.550      0.584      -0.211       0.120
==============================================================================
Omnibus:                       62.028   Durbin-Watson:                   1.405
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              353.003
Skew:                           2.067   Prob(JB):                     2.22e-77
Kurtosis:                      11.602   Cond. No.                     1.33e+05
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [9]:
import time
import itertools

In [14]:
def processSubset(X, y, feature_set):
    model = sm.OLS(y, X[list(feature_set)])
    regr = model.fit()
    AIC = regr.aic
    return {'model' : regr, 'AIC' : AIC}

def forward(X, y, predictors):
    remaining_predictors = [p for p in X.columns.difference(['Intercept']) if p not in predictors]
    results = []
    for p in remaining_predictors:
        results.append(processSubset(X = X, y = y, feature_set = predictors + [p] + ['Intercept']))
        
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['AIC'].argmin()]
    
    print('Processed ', models.shape[0], 'models on', len(predictors) + 1, 'predictors in')
    print('Selected predictors: ', best_model['model'].model.exog_names, ' AIC:', best_model[0])
    return best_model

def backward(X, y, predictors):
    tic = time.time()
    results = []
    
    for combo in itertools.combinations(predictors, len(predictors) -1):
        results.append(processSubset(X = X, y = y, feature_set = list(combo) + ['Intercept']))
    models = pd.DataFrame(results)
    
    best_model = models.loc[models['AIC'].argmin()]
    toc = time.time()
    print('Processed ', models.shape[0], 'models on', len(predictors) - 1, 'predictors in', (toc - tic))
    print('Selected predictors:', best_model['model'].model.exog_names, ' AIC:', best_model[0])
    return best_model

def Stepwise_model(X, y):
    Stepmodels = pd.DataFrame(columns = ['AIC', 'model'])
    tic = time.time()
    predictors = []
    Smodel_before = processSubset(X, y, predictors + ['Intercept'])['AIC']
    for i in range(1, len(X.columns.difference(['Intercept'])) + 1):
        Forward_result = forward(X=X, y=y, predictors=predictors) # constant added
        print('forward')
        Stepmodels.loc[i] = Forward_result
        predictors = Stepmodels.loc[i]["model"].model.exog_names
        predictors = [ k for k in predictors if k != 'Intercept']
        Backward_result = backward(X=X, y=y, predictors=predictors)
        if Backward_result['AIC']< Forward_result['AIC']:
            Stepmodels.loc[i] = Backward_result
            predictors = Stepmodels.loc[i]["model"].model.exog_names
            Smodel_before = Stepmodels.loc[i]["AIC"]
            predictors = [ k for k in predictors if k != 'Intercept']
            print('backward')
        if Stepmodels.loc[i]['AIC']> Smodel_before:
            break
        else:
            Smodel_before = Stepmodels.loc[i]["AIC"]
    toc = time.time()
    print("Total elapsed time:", (toc - tic), "seconds.")
    return (Stepmodels['model'][len(Stepmodels['model'])])   

In [15]:
from patsy import dmatrices

y, X = dmatrices('Price ~ EngineSize + RPM + Weight + Length', data = Cars, return_type = 'dataframe')

In [16]:
Stepwise_best_model = Stepwise_model(X = X, y = y)

Processed  4 models on 1 predictors in
Selected predictors:  ['Weight', 'Intercept']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002B5DC6665E0>
forward
Processed  1 models on 0 predictors in 0.004996538162231445
Selected predictors: ['Intercept']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002B5DC65C400>
Processed  3 models on 2 predictors in
Selected predictors:  ['Weight', 'RPM', 'Intercept']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002B5DC666490>
forward
Processed  2 models on 1 predictors in 0.005995512008666992
Selected predictors: ['Weight', 'Intercept']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002B5C8E544F0>
Processed  2 models on 3 predictors in
Selected predictors:  ['Weight', 'RPM', 'EngineSize', 'Intercept']  AIC: <statsmodels.regression.linear_model.RegressionResultsWrapper object at 0x000002B5DC666760>
forward
Pro

In [17]:
Stepwise_best_model

In [18]:
Stepwise_best_model.aic

616.0976497740975

In [19]:
Stepwise_best_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  Price   R-squared:                       0.561
Model:                            OLS   Adj. R-squared:                  0.547
Method:                 Least Squares   F-statistic:                     37.98
Date:                Mon, 04 Apr 2022   Prob (F-statistic):           6.75e-16
Time:                        22:26:31   Log-Likelihood:                -304.05
No. Observations:                  93   AIC:                             616.1
Df Residuals:                      89   BIC:                             626.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Weight         0.0073      0.002      3.372      0.001       0.003       0.012
RPM            0.0071      0.001      5.208      0.000       0.004       0.010
EngineSize     4.3054      1.325      3.249      0.002       1.673       6.938
Intercept    -51.7933      9.106     -5.688      0.000     -69.887     -33.699
==============================================================================
Omnibus:                       62.441   Durbin-Watson:                   1.406
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              361.880
Skew:                           2.076   Prob(JB):                     2.62e-79
Kurtosis:                      11.726   Cond. No.                     8.27e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 8.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""